# Training

In [1]:
#immport necessary libraries
import os
import numpy as np
import torch
from PIL import Image

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

from engine import train_one_epoch, evaluate, validate
import utils
import torchvision.transforms as tra
from torchvision.transforms import functional as F
import matplotlib.pyplot as plt
import cv2
#import generate_masks as gm
import random

## Load up the model

In [2]:

#define the number of class
num_classes=2 #background and grape
# load an instance segmentation model pre-trained pre-trained on COCO
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
# now get the number of input features for the mask classifier
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
# and replace the mask predictor with a new one
model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                    hidden_layer,
                                                    num_classes)

## Create the Dataset Object and dataloader

In [3]:
import torchvision.datasets as dset
import albumentations as A
from datatools import COCOInstanceSegmentationDataset, get_mask_transform


# the dataset object
img_path="./images"    #this is the path to your images
json_path="./trainval_instance_segmentation.json"  #this is the path to your json

#you can add your own albumentations transforms here 
transform_list = A.Compose([
#A.Resize(200, 300),
#A.CenterCrop(100, 100),
#A.RandomCrop(80, 80),
A.HorizontalFlip(p=0.5),
A.Rotate(limit=(-6, 6), p=0.6),
#A.ShiftScaleRotate(rotate_limit=50, p=0.6)
#A.SmallestMaxSize(max_size=1292, p=1)
#A.augmentations.transforms.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.15, rotate_limit=5, p=0.5)
A.RandomScale(scale_limit=0.2, p=0.5)
#A.core.composition.OneOf([A.Resize(2298, 1292, interpolation=1, always_apply=False, p=1), A.Resize(3109, 1748, interpolation=1, always_apply=False, p=1), A.Resize(2568, 1444, interpolation=1, always_apply=False, p=1), A.Resize(2839, 1596, interpolation=1, always_apply=False, p=1)],p=0.5)
#A.augmentations.transforms.Resize(2298, 1292, interpolation=1, always_apply=False, p=1)
#A.VerticalFlip(p=0.5),
#A.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainval = COCOInstanceSegmentationDataset(root = img_path,json_dir = json_path, transforms=transform_list, train=True)


train, val = torch.utils.data.random_split(trainval, [2, 1]) # put 2 images in train and 1 in val for this dummy example



# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(train, batch_size=1, shuffle=True, num_workers=0, collate_fn=utils.collate_fn)


data_loader_validate = torch.utils.data.DataLoader(train, batch_size=1, shuffle=False, num_workers=0, collate_fn=utils.collate_fn)
    

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


## Now lets train for 50 epochs

In [4]:
# now start the training process
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.009, momentum=0.9, weight_decay=0.0006)

# and a learning rate scheduler (optional)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=1)  # optional lr scheudler (gamma is 1 by default which means no scheduling)

# let's train it for 50 epochs and save after best epoch
num_epochs = 50

best_val_loss=999
for epoch in range(num_epochs):
    model.train()
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=2)
    # update the learning rate (optional)
    lr_scheduler.step()
    # evaluate and checkpoint using the validate dataset
    best_val_loss=validate(model, data_loader_validate, device=device, best_val_loss=best_val_loss)


print('Trained for '+str(num_epochs) + ' epochs')

/home/pvraja/miniconda3/envs/greenhouse/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0]  [0/2]  eta: 0:00:02  lr: 0.009000  loss: 6.5906 (6.5906)  loss_classifier: 0.6580 (0.6580)  loss_box_reg: 0.3632 (0.3632)  loss_mask: 1.6684 (1.6684)  loss_objectness: 3.5628 (3.5628)  loss_rpn_box_reg: 0.3382 (0.3382)  time: 1.4230  data: 1.2495  max mem: 1153
Epoch: [0]  [1/2]  eta: 0:00:01  lr: 0.009000  loss: 5.8709 (6.2308)  loss_classifier: 0.6467 (0.6524)  loss_box_reg: 0.3632 (0.4592)  loss_mask: 1.6684 (2.0077)  loss_objectness: 2.1773 (2.8701)  loss_rpn_box_reg: 0.1447 (0.2415)  time: 1.0884  data: 0.9327  max mem: 1467
Epoch: [0] Total time: 0:00:02 (1.0905 s / it)
Best Model Saved! loss: tensor(11.8631, device='cuda:0')
Epoch: [1]  [0/2]  eta: 0:00:01  lr: 0.009000  loss: 5.6280 (5.6280)  loss_classifier: 0.2242 (0.2242)  loss_box_reg: 0.1870 (0.1870)  loss_mask: 4.4947 (4.4947)  loss_objectness: 0.5419 (0.5419)  loss_rpn_box_reg: 0.1802 (0.1802)  time: 0.8008  data: 0.6418  max mem: 1467
Epoch: [1]  [1/2]  eta: 0:00:01  lr: 0.009000  loss: 2.6957 (4.1618)  loss